In [1]:
import pandas as pd
from numpy import random
from faker import Faker

In [22]:
trans = pd.read_json('../00_data/transition_matrix.json')
trans

,checkout,dairy,drinks,fruit,spices
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.393033,0.000000,0.222483,0.189357,0.195127
drinks,0.537260,0.027145,0.000000,0.218950,0.216645
fruit,0.500195,0.237993,0.136080,0.000000,0.125732
spices,0.251998,0.323122,0.272776,0.152104,0.000000


In [23]:
trans.loc['dairy', ]

checkout    0.393033
dairy       0.000000
drinks      0.222483
fruit       0.189357
spices      0.195127
Name: dairy, dtype: float64

In [24]:
initial_condition = pd.read_json('../00_data/initial_condition.json',typ='Series')
initial_condition

checkout    0.000000
dairy       0.312085
drinks      0.159363
fruit       0.304117
spices      0.224436
dtype: float64

In [15]:
random.choice(initial_condition.index.values, p = initial_condition)

'fruit'

In [232]:


class Customer:
    """
    a single customer that moves through the supermarket
    in a MCMC simulation
    """
    ...
    def __init__(self, initial_state_probabilities, transition_matrix, deltat, budget=100):
        fk = Faker()
        
        self.name =  fk.name()
        self.budget = budget
        self.initial_state_probabilities = initial_state_probabilities
        self.state = random.choice(initial_state_probabilities.index.values, p = initial_state_probabilities)
        self.transition_matrix = transition_matrix
        self.steps = [self.state]
        self.deltat=deltat
        
    
    def __repr__(self):
        return f'<Customer {self.name} in {self.state}>'

    def next_state(self):
        '''
        Propagates the customer to the next state.
        Returns nothing.
        '''
        choices = self.transition_matrix.index.values
        probs = self.transition_matrix.loc[self.state]
        # print(choices, probs)
        self.state = random.choice(choices, p = probs)
        self.steps.append(self.state)

    def is_active(self):
        """Returns True if the customer has not reached the checkout yet."""
        if self.state!='checkout':
            return True
        else:
            return False

    def sim(self):
        self.next_state()
        while self.is_active():
            self.next_state()

    def get_time(self):
        assert isinstance(self.deltat, pd._libs.tslibs.timedeltas.Timedelta), 'Need a time delta'
        if not self.is_active():
            timedf = pd.DataFrame({'steps' : self.steps, 'time' : [self.deltat * v for v in range(len(self.steps))]})
            return timedf
        else:
            return None

        



In [233]:
c1 = Customer(transition_matrix=trans,
        initial_state_probabilities=initial_condition,
        deltat=pd.Timedelta(5, 'minutes'))
print(c1)

c1.sim()
print(c1.steps)
print(c1)
c1.get_time()

<Customer Elizabeth Hahn in dairy>
['dairy', 'spices', 'drinks', 'checkout']
<Customer Elizabeth Hahn in checkout>


,steps,time
0,dairy,0 days 00:00:00
1,spices,0 days 00:05:00
2,drinks,0 days 00:10:00
3,checkout,0 days 00:15:00


[Timedelta('0 days 00:00:00'),
 Timedelta('0 days 00:05:00'),
 Timedelta('0 days 00:10:00'),
 Timedelta('0 days 00:15:00'),
 Timedelta('0 days 00:20:00')]

next steps:

 - add time delta for each step (by distance between sections? use dot product to calculate?)
 - simulate for many

In [236]:
n_customers = 1000
# customers = [Customer(transition_matrix=trans, initial_state_probabilities=initial_condition) for i in range(n_customers)]
customers = [None] * n_customers

for i in range(n_customers):
    customers[i] = Customer(transition_matrix=trans, initial_state_probabilities=initial_condition,deltat= pd.Timedelta(5, 'minutes'))



In [237]:
customer_steps = [None] * n_customers

for idx, c in enumerate(customers):
    c.sim()
    customer_steps[idx] = c.steps

In [238]:
for j in range(10):
    print(customers[j], customers[j].steps)

<Customer David Rowland in checkout> ['spices', 'dairy', 'fruit', 'dairy', 'drinks', 'fruit', 'checkout']
<Customer Brittney Campbell in checkout> ['dairy', 'fruit', 'dairy', 'drinks', 'checkout']
<Customer Christopher Baldwin in checkout> ['drinks', 'checkout']
<Customer Steven Nelson in checkout> ['dairy', 'checkout']
<Customer Carolyn Brady in checkout> ['drinks', 'checkout']
<Customer Katherine Grant in checkout> ['dairy', 'fruit', 'dairy', 'checkout']
<Customer Jeffrey Burke in checkout> ['drinks', 'spices', 'drinks', 'checkout']
<Customer Miranda Gordon in checkout> ['fruit', 'checkout']
<Customer Clinton Griffin in checkout> ['drinks', 'fruit', 'spices', 'drinks', 'fruit', 'dairy', 'fruit', 'drinks', 'checkout']
<Customer Cindy West in checkout> ['spices', 'checkout']


In [239]:
pd.to_datetime("2022-07-06")

Timestamp('2022-07-06 00:00:00')

## With supermarket class

In [ ]:
class supermarket(opening, closing, entry_rate, super_deltat, customer_deltat):
    
    # entry rate and super delta t must have same temporal resoltion, use hourly

    # cycle from opening to closing by super_delta t

    # for every step, add n customers at a random entry time

    # execute sim calculate sim time + entry time

    # at last step, check if any customers extend past closing time.
    # drop all steps at or past closing time, and add a checkout at closing time instead